# Import modules

In [1]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm, block_diag
from scipy.constants import h,k, hbar
from math import pi
from qutip import *
import sympy
import functools, operator
from collections import OrderedDict
import itertools

from transmonCPW import Transmon

Using matplotlib backend: TkAgg


# Class/function

In [2]:
# import numpy as np

def compute_g_eff(wc, w1, w2, C1, C2, C12, C1c, C2c, Cc):
    """
    w in GHz and C in fF
    """
    eta = C1c*C2c/C12/Cc
    Sigma = 2 / (1/(w1+wc) + 1/(w2+wc))
    Delta = 2/(1/(w1-wc) + 1/(w2-wc))

    g = 1/2*(wc*eta/2/Delta - wc*eta/2/Sigma + eta + 1)* C12/np.sqrt(C1*C2)*np.sqrt(w1*w2)
    
    return g

def compute_g1(wc,w1,C1,C1c,Cc):
    """
    Qubit-coupler coupling strengh in GHz.
    C in fF
    """
    g1 = 1/2*C1c/np.sqrt(C1*Cc)*np.sqrt(w1*wc)
    
    return g1

def compute_g2(wc,w2,C2,C2c,Cc):
    """
    Qubit-coupler coupling strengh in GHz.
    C in fF
    """
    g2 = 1/2*C2c/np.sqrt(C2*Cc)*np.sqrt(w2*wc)
    
    return g2

def compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc):
    """
    Qubit-qubit coupling strengh in GHz.
    C in fF
    """
    eta = C1c*C2c/C12/Cc
    g12 = 1/2*(1 + eta)*C12/np.sqrt(C1*C2)*np.sqrt(w1*w2)
    
    return g12

def hamiltonian_2Q_coupler(g1, g2, g12, w1,w2,wc,alpha1,alpha2,alphac,N):
    """
    Full Hamiltonian of two transmons and a coupler.
    Note: This produces negative ZZ as experimentally shown in Li paper.
    """
#     g1 = compute_g1(wc,w1,C1,C1c,Cc)
#     g2 = compute_g2(wc,w2,C2,C2c,Cc)
#     g12 = compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)

    b = destroy(N)
    I = qeye(N)
    
    # Eq. A17
    H1 = w1*tensor(b.dag()*b, I,I) + alpha1/2.0*tensor(b.dag()*b.dag()*b*b, I, I)
    H2 = w2*tensor(I, b.dag()*b, I) + alpha2/2.0*tensor(I, b.dag()*b.dag()*b*b, I)
    Hc = wc*tensor(I, I, b.dag()*b) + alphac/2.0*tensor(I, I, b.dag()*b.dag()*b*b)
    H1c = g1*(tensor(b.dag(), I, b) + tensor(b, I, b.dag())- tensor(b.dag(), I, b.dag()) -tensor(b, I, b))
    H2c = g2*(tensor(I, b.dag(), b) + tensor(I, b, b.dag())- tensor(I, b.dag(), b.dag()) -tensor(I, b, b))
    H12 = g12*(tensor(b.dag(), b, I) + tensor(b, b.dag(), I) - tensor(b.dag(), b.dag(), I) -tensor(b, b, I))
    
    H_eff = H1 + H2 + Hc + H1c + H2c + H12
    
    return H_eff

# Transmon params

In [3]:
def print_result(q1):
    l = [q1.total_cap/1e-15, q1.anharmonicity/1e6, q1.Ej_to_Ec_ratio, q1.f01/1e9]
    print(f'Ej = {q1.Ej/h/1e9:.3g} GHz')
    print(f'Ec = {q1.Ec/h/1e9:.3g} GHz')
    print(f'Total Cap = {q1.total_cap/1e-15:.3g} fF')
    print(f'Anharm. = {q1.anharmonicity/1e6:.3g} MHz')
    print(f'Ej/Ec = {q1.Ej_to_Ec_ratio:.3g}')
    print(f'f01 = {q1.f01/1e9:.3g} GHz')
    print()
    
q1 = Transmon.in_GHz(13, 0.28)
q2 = Transmon.in_GHz(22, 0.3)

print_result(q1)
print_result(q2)

Ej = 13 GHz
Ec = 0.28 GHz
Total Cap = 69.2 fF
Anharm. = -324 MHz
Ej/Ec = 46.4
f01 = 5.1 GHz

Ej = 22 GHz
Ec = 0.3 GHz
Total Cap = 64.6 fF
Anharm. = -335 MHz
Ej/Ec = 73.3
f01 = 6.95 GHz



In [4]:
T_eff = 0.1
f_01 = 5e9 
p_1 = np.exp(-h*f_01/(k*T_eff))/(np.exp(-h*f_01/(k*T_eff))+1)
print(f'f_01 = {f_01/1e9:.3f} GHz')
print(f'T_eff = {T_eff:.3f} K')
print(f'|1> state population ={p_1:.3f}')

f_01 = 5.000 GHz
T_eff = 0.100 K
|1> state population =0.083


# (g_eff, g1, g12) vs coupler freq.

In [5]:
# parameters from Yan's paper
# w1, w2 = 4, 4 # in GHz
# (C1, C2, C12, C1c, C2c, Cc) = (70,72,0.1,4,4.2,200)

# parameters for IBM qubit
w1, w2 = 5, 5 # in GHz
(C1, C2, C12, C1c, C2c, Cc) = (60,62,0.1,2,2.2,50)
# (C1, C2, C12, C1c, C2c, Cc) = (60,60,0.10,10,14,100)

wc_list = np.linspace(5.5, 8, 101)
geff = compute_g_eff(wc_list,w1,w2,C1, C2,C12,C1c,C2c,Cc)
g1 = compute_g1(wc_list,w1,C1,C1c,Cc)
g12 = compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)

fig,(ax,ax1) = plt.subplots(2,1, figsize=(10,8))
ax.plot(wc_list, geff*1e3)
ax.set_ylabel(r'$g_{eff}$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

ax1.plot(wc_list, g1*1e3)
ax1.set_xlabel('Coupler Frequency (GHz)', fontsize=18)
ax1.set_ylabel(r'$g_{12}$ (MHz)', fontsize=18)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.grid('on')

print(f'g12 = {compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3:.3f} MHz')

g12 = 7.706 MHz


# Two transmons + Coupler

In [6]:
# parameters for IBM qubit
w1, w2, wc = 5, 5, 6 # in GHz
alpha1, alpha2, alphac = -0.35, -0.35, -0.35# in GHz
(C1, C2, C12, C1c, C2c, Cc) = (70,70,0.1,2,2,60)
N=3

g1 = compute_g1(wc,w1,C1,C1c,Cc)
g2 = compute_g2(wc,w2,C2,C2c,Cc)
g12 = compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)  
H_eff2 = hamiltonian_2Q_coupler(g1, g2, g12,w1,w2,wc,alpha1,alpha2,alphac,N)

eigen2 = H_eff2.eigenstates()
ZZ = eigen2[0][6] - eigen2[0][1] - eigen2[0][2] + eigen2[0][0]   # in GHz

sympy.SparseMatrix(np.round((H_eff2)[:, 0:13],2))
print(f'g1={compute_g1(wc,w1,C1,C1c,Cc)*1e3:.1f} MHz')
print(f'g12={compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3:.3f} MHz')
print(f'geff={compute_g_eff(wc,w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3:.3f} MHz')
print(f'ZZ = {ZZ*1e3:.5f} (MHz)')

g1=84.5 MHz
g12=5.952 MHz
geff=-1.840 MHz
ZZ = -0.05389 (MHz)


In [7]:
H_eff2

Quantum object: dims = [[3, 3, 3], [3, 3, 3]], shape = (27, 27), type = oper, isherm = True
Qobj data =
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -8.45154255e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -8.45154255e-02  0.00000000e+00
  -5.95238095e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  6.00000000e+00  0.00000000e+00  8.45154255e-02
   0.00000000e+00 -1.19522861e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.45154255e-02  0.00000000e+00 -1.19522861e-01
   0.00000000e+00 -5.95238095e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [

In [8]:
# show H_eff2 matrix
# sympy.SparseMatrix(np.round((H_eff2)[:, 0:13],2))

In [9]:
# plot_energy_levels([H_eff2], N=7, show_ylabels=True)

## (ZZ, g) vs Coupler frequency

In [10]:
newplot = True

# parameters for IBM qubit
w1, w2 = 5.3, 5.15 # in GHz
wc = 5.5
# (C1, C2, C12, C1c, C2c, Cc) = (70,70,0.1, 2, 2, 65) # seems good
(C1, C2, C12, C1c, C2c, Cc) = (70,70,0.1, 2, 2, 65)
alpha1 = -0.35 # in GHz
alpha2 = -0.35
alphac = -0.3

# # parameters for Li's paper
# w1, w2 = 4.961, 4.926 # in GHz
# wc = 5.977
# (C1, C2, C12, C1c, C2c, Cc) = (60,60,0.13, 2.4,2.4, 90)
# alpha1 = -0.206 # in GHz
# alpha2 = -0.202
# alphac = -0.254

N = 3

# sweep
wc_list = np.linspace(5.5,7,101)
# wc_list = np.linspace(5.35,6.2,101)
ZZ_list, g_eff_list, g1_list, g2_list, g12_list = [], [], [], [], []
ev_list = []

for wc in wc_list:
    g1 = compute_g1(wc,w1,C1,C1c,Cc)
    g2 = compute_g2(wc,w2,C2,C2c,Cc)
    g12 = compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)  
    g_eff = compute_g_eff(wc,w1,w2,C1,C2,C12,C1c,C2c,Cc)
    H_eff = hamiltonian_2Q_coupler(g1,g2,g12,w1,w2,wc,alpha1,alpha2,alphac,N)
    
    eigen = H_eff.eigenstates()
    ev_list.append(eigen[1][6])
    ZZ = (eigen[0][6] - eigen[0][1] - eigen[0][2] + eigen[0][0])*1e3  # in MHz
    
    ZZ_list.append(ZZ)
    g_eff_list.append(g_eff*1e3)
    g1_list.append(g1*1e3)
    g2_list.append(g2*1e3)
    g12_list.append(g12*1e3)
   
# plot
if newplot:
    fig,(ax,ax1, ax2) = plt.subplots(3,1, figsize=(8,8))
ax.plot(wc_list, ZZ_list,'-')
ax.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

ax1.plot(wc_list, g_eff_list,'-')
ax1.plot(wc_list, g12_list,'-')
ax1.set_ylabel(r'$g_{eff}, g_{12}$ (MHz)', fontsize=18)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.grid('on')

ax2.plot(wc_list, g1_list,'-')
ax2.plot(wc_list, g2_list,'-')
ax2.set_xlabel('Coupler Frequency (GHz)', fontsize=18)
ax2.set_ylabel(r'$g_1, g_2$ (MHz)', fontsize=18)
ax2.tick_params(axis='x', labelsize=18)
ax2.tick_params(axis='y', labelsize=18)
ax2.grid('on')

In [11]:
max_arg = []
for ev in ev_list:
    max_arg.append(np.argmax(np.absolute(ev.full())))
# max_arg

## Find Pauli Coeffi

In [12]:
# parameters for IBM qubit
w1, w2 = 5.3, 5.15 # in GHz
wc = 5.5
(C1, C2, C12, C1c, C2c, Cc) = (60,60,0.1, 2,2, 100)
alpha1 = -0.35 # in GHz
alpha2 = -0.35
alphac = -0.2

N = 3

# Hamiltonian
H_eff = hamiltonian_eff_2Q_coupler(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,alphac,N)

# By solving eigenvalue
H1 = H0 + V0
eigen = H1.eigenstates()

# find U
U = sorted_diag_op(eigen)
U.dims = H1.dims

# digonalize H0
H2 = U.dag() * H1 * U

NameError: name 'hamiltonian_eff_2Q_coupler' is not defined

# ZZ by effective two-qubit Hamiltonian
Ref: Yan et al., PRApplied 2018

In [ ]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from qutip import *
import sympy

In [13]:
def hamiltonian_eff_two_qubit(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,N):
    """
    Effective Two-qubit Hamiltonian, coupler decoupled.
    """
    b1 = destroy(N)
    b2 = destroy(N)

    w1_dr = w1 + compute_g1(wc,w1,C1,C1c,Cc)**2 * (1/(w1-wc)-1/(w1+wc))
    w2_dr = w2 + compute_g2(wc,w2,C1,C1c,Cc)**2 * (1/(w2-wc)-1/(w2+wc))
    geff = compute_g_eff(wc, w1, w2, C1, C2, C12, C1c, C2c, Cc)

    # Eq. B2
    H_eff = ( w1_dr*tensor(b1.dag()*b1,qeye(N)) + alpha1/2.0*tensor(b1.dag()*b1.dag()*b1*b1, qeye(N))
        + w2_dr*tensor(qeye(N),b2.dag()*b2) + alpha2/2.0*tensor(qeye(N), b2.dag()*b2.dag()*b2*b2)
        + geff * (tensor(b1.dag(),b2) + tensor(b1, b2.dag())))

    return H_eff

In [14]:
# parameters for IBM qubit
w1, w2 = 5.29, 5.16 # in GHz
wc = 6
(C1, C2, C12, C1c, C2c, Cc) = (70,72,0.1,4,4.2,200)

alpha1 = -0.35 # in GHz
alpha2 = -0.35

N = 3

# compute Hamiltonian
H_eff1 = hamiltonian_eff_two_qubit(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,N)
eigen1 = H_eff1.eigenstates()

sympy.SparseMatrix(np.round((H_eff1)[:, 0:13],2))
print(f'g1={compute_g1(wc,w1,C1,C1c,Cc)*1e3:.1f} MHz')
print(f'g12={compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3:.3f} MHz')
print(f'geff={compute_g_eff(wc,w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3:.3f} MHz')

# Compute eigenvalues and find $\zeta = \omega_{11}-\omega_{01}-\omega_{10}$
ZZ = eigen1[0][5] - eigen1[0][1] - eigen1[0][2] + eigen1[0][0]  # in GHz
print(f'ZZ = {ZZ*1e3:.2f} (MHz)')

g1=95.2 MHz
g12=6.771 MHz
geff=-6.105 MHz
ZZ = 0.49 (MHz)


In [15]:
# compute eigenvalues and eigenstates.
eigen1 = H_eff1.eigenstates()  # eigen is a tuble with two elements. eigen[0] = eigenvalues, eigen[1]=eigenstates.
eigen1[0]

array([ 0.        ,  5.14838566,  5.27671523,  9.94719744, 10.20251379,
       10.42559143, 15.22262105, 15.35268161, 20.15020177])

## (ZZ, g_eff) vs Coupler Frequency

In [16]:
# Note: This diagonal effective Hamiltonian doesn't seem to give right ZZ. This is
# probabaly because SW transformation was used up to only 2nd order. It looks like
# 4th order approximation is necessary. So, the only full Hamiltonian with 
# g coupling terms would work to give right ZZ.
# However, for driven ZZ calculation, it may be ok to use the digonal effective 
# two-quibt Hamiltonian? I want to know the sign and magnitude of 
# driven ZZ vs CR ampliutude. 
# If I can do this with two-transmon + bus cavity case, then I could do the same thing 
# tunable coupler case since the effective two-qubit Hamiltonian for coupler case has
# the same form as bus cavity case. The only difference is J vs g_eff. If I assume, 
# ZZ goes roughly by J**2, then driven ZZ would just increase...?

# parameters for IBM qubit
w1, w2 = 5.29, 5.16 # in GHz
wc = 5.5
(C1, C2, C12, C1c, C2c, Cc) = (60,60,0.02,1,1, 50)
alpha1 = -0.35 # in GHz
alpha2 = -0.35

N = 3

# sweep
wc_list = np.linspace(5.4,7.5,31)
ZZ_list, g_eff_list, g1_list, g2_list, g12_list = [], [], [], [], []
ev_list = []

for wc in wc_list:
    H_eff = hamiltonian_eff_two_qubit(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,N)
    eigen = H_eff.eigenstates()
    ev_list.append(eigen[1][5])
    ZZ = (eigen[0][5] - eigen[0][1] - eigen[0][2] + eigen[0][0])*1e3  # in MHz
    
    ZZ_list.append(ZZ)
    g_eff_list.append(compute_g_eff(wc,w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3)
    g1_list.append(compute_g1(wc,w1,C1,C1c,Cc)*1e3)
    g2_list.append(compute_g2(wc,w2,C2,C2c,Cc)*1e3)
    g12_list.append(compute_g12(w1,w2,C1,C2,C12,C1c,C2c,Cc)*1e3)
    
   
# plot
fig,(ax,ax1, ax2) = plt.subplots(3,1, figsize=(10,10))
ax.plot(wc_list, ZZ_list,'-')
# ax.set_xlabel('Coupler Frequency (GHz)', fontsize=18)
ax.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

ax1.plot(wc_list, g_eff_list,'-')
ax1.plot(wc_list, g12_list,'-')
ax1.set_ylabel(r'$g_{eff}, g_{12}$ (MHz)', fontsize=18)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.grid('on')

ax2.plot(wc_list, g1_list,'-')
ax2.plot(wc_list, g2_list,'-')
ax2.set_xlabel('Coupler Frequency (GHz)', fontsize=18)
ax2.set_ylabel(r'$g_1, g_2$ (MHz)', fontsize=18)
ax2.tick_params(axis='x', labelsize=18)
ax2.tick_params(axis='y', labelsize=18)
ax2.grid('on')

In [17]:
max_arg = []
for ev in ev_list:
    max_arg.append(np.argmax(np.absolute(ev.full())))

In [18]:
eigen[1][5].full()

array([[0.00000000e+00+0.j],
       [0.00000000e+00+0.j],
       [1.42675092e-04+0.j],
       [0.00000000e+00+0.j],
       [9.99999941e-01+0.j],
       [0.00000000e+00+0.j],
       [3.11040307e-04+0.j],
       [0.00000000e+00+0.j],
       [0.00000000e+00+0.j]])

# Compare energy levels of eff. Hamil. vs diag. eff. Hamil.

In [19]:
# parameters for IBM qubit
w1, w2 = 5, 5 # in GHz
wc = 5.5
(C1, C2, C12, C1c, C2c, Cc) = (70,72,0.1,4,4.2,150)
alpha1 = -0.35 # in GHz
alpha2 = -0.35
alphac = -0.4

N = 5
eigen1 = hamiltonian_eff_two_qubit(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,N).eigenstates()
eigen2 = hamiltonian_eff_2Q_coupler(w1,w2,wc,C1,C2,C12,C1c,C2c,Cc,alpha1,alpha2,alphac,N).eigenstates()

plt.figure()
plt.plot(eigen1[0],'-o', label='Effective two-transmons Hamil.')
plt.plot(eigen2[0],'-o', label='Two-qubit+ coupler Hamil.')
plt.legend()
plt.grid()

NameError: name 'hamiltonian_eff_2Q_coupler' is not defined